In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
import tensorflow as tf

In [4]:
data = pd.read_excel("/content/drive/MyDrive/Code/AI/KAN/7_Classification_Done/Dry_Bean_Dataset/Dry_Bean_Dataset.xlsx")
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [5]:
data["Class"].value_counts()

Class
DERMASON    3546
SIRA        2636
SEKER       2027
HOROZ       1928
CALI        1630
BARBUNYA    1322
BOMBAY       522
Name: count, dtype: int64

In [6]:
data.shape

(13611, 17)

In [7]:
data.isna().sum()

Area               0
Perimeter          0
MajorAxisLength    0
MinorAxisLength    0
AspectRation       0
Eccentricity       0
ConvexArea         0
EquivDiameter      0
Extent             0
Solidity           0
roundness          0
Compactness        0
ShapeFactor1       0
ShapeFactor2       0
ShapeFactor3       0
ShapeFactor4       0
Class              0
dtype: int64

In [8]:
features = data.drop("Class", axis=1)
target = data["Class"]

In [9]:
target

0           SEKER
1           SEKER
2           SEKER
3           SEKER
4           SEKER
           ...   
13606    DERMASON
13607    DERMASON
13608    DERMASON
13609    DERMASON
13610    DERMASON
Name: Class, Length: 13611, dtype: object

In [10]:
le = LabelEncoder()
target_encoded = le.fit_transform(target)
target_encoded

scaler = StandardScaler() # .set_output(transform="pandas")
features_scaled = scaler.fit_transform(features)
features_scaled

array([[-0.84074853, -1.1433189 , -1.30659814, ...,  2.40217287,
         1.92572347,  0.83837103],
       [-0.82918764, -1.01392388, -1.39591111, ...,  3.10089314,
         2.68970162,  0.77113842],
       [-0.80715717, -1.07882906, -1.25235661, ...,  2.23509147,
         1.84135576,  0.91675514],
       ...,
       [-0.37203825, -0.44783294, -0.45047814, ...,  0.28920441,
         0.33632829,  0.39025114],
       [-0.37176543, -0.42702856, -0.42897404, ...,  0.22837538,
         0.2489734 ,  0.03644001],
       [-0.37135619, -0.38755718, -0.2917356 , ..., -0.12777587,
        -0.2764814 ,  0.71371948]])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled,
                                                    target_encoded,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=target_encoded)

In [12]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6])

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size=32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size=32)

In [ ]:
model = Sequential(name="DryBeansModel")
model.add(Input(shape=(16,)))
model.add(Dense(20, activation="relu"))
model.add(Dense(15, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(7, activation="softmax"))

optimizer = Adam(learning_rate=0.0002)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
history=model.fit(X_train, y_train, epochs=200, batch_size=64,
                  validation_data=(X_test, y_test))#,callbacks=[early_stopping])

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('\nTest accuracy:', accuracy)

86/86 [==============================] - 0s 2ms/step - loss: 0.2054 - acc: 0.9232

Test accuracy: 0.923246443271637
